# 物体運動のシミュレーション

宇都宮大学　吉田勝俊

**学習内容**

1. 運動と運動方程式
1. 微分方程式の数値計算
1. 数値解のアニメーション
1. 大気中の質点の放物運動
1. 運動の設計（≒条件の調整）

***
※ Python では，必要な機能を「インポート」して使います．

In [ ]:
import numpy as np                              #数値計算機能のインポート 別名np
import matplotlib.pyplot as plt                 #グラフ描画機能のインポート 別名plt
from scipy.integrate import odeint              #常微分方程式の数値解法をインポート
from matplotlib.animation import FuncAnimation  #アニメーション機能のインポート
from matplotlib import rc                       #各種設定機能のインポート
rc('animation', html='jshtml')                  #Colabでアニメーション表示可能にするための設定

***
## §１ 運動と運動方程式

* 運動　⇔　物体の位置 $x$ の時間変化 $x(t)$ のこと！
* 運動方程式　⇔　物体の運動 $x(t)$ を求める方程式のこと！

### 高校物理の運動方程式

$ma(t) = f$　　（質量）×（加速度）＝（力）


| 記号 | 名称 | 説明 |
|:------|:------|:------|
| $m$  | 質量 | 運動（位置の時間変化）ではない |
| $f$  | 力 | 同上 |
| $a(t)$  | 加速度 | 同上 |

**結論**
* 高校の運動方程式を解いても，「加速度」しか求まらない．
* 運動方程式なのに，「運動」は求まらない！

### 大学物理の運動方程式

$m x''(t) = f$　　（質量）×（加速度）＝（力）

| 記 号 | 名 称 | 説 明 |
| ------ | ------ | ------ |
| $m$  | 質量 | 運動（位置の時間変化）ではない |
| $f$  | 力 | 同上 |
| $x(t)$  | 運動 | 位置の時間変化！ |

**微分法**

| &emsp;&emsp;&emsp;&emsp;&nbsp;記 号&nbsp;&emsp;&emsp;&emsp;&emsp; | 名 称 | 説 明 | 数学 |
| ------ | ------ | ------ | ------ |
| $x(t)$  | 運動 | 位置の時間変化 |
| $v(t)=x'(t)$  | 速度 | 位置 $x(t)$ の<br>グラフの傾き| 運動の微分 |
| $a(t)=v'(t)=x''(t)$  | 加速度 | 速度 $v(t)$ の<br>グラフの傾き | 速度の微分 |

* $a(t)=v'(t)=\fbox{$x'(t)$}'=x''(t)$

**結論**
* 大学の運動方程式を解くと，「運動」が求まる！

### 運動方程式の解き方

* 手計算
  * 長所・・・運動が数式で求まる　→条件の逆算（設計という）がしやすい
  * 短所・・・対象がちょっとでも複雑だと解けない

* 数値解法（コンピュータによる近似計算のこと）
  * 短所・・・運動は数式ではなく，数列として求まる
  * 長所・・・ロボットだろうが，スペースシャトルだろうが解ける

> #### ★機械構造としては，超シンプルなのに，手計算では解けない運動方程式の例
>
> <img src="https://github.com/ktysd/outreach/raw/main/docs/invpend.png" width="20%" valign="middle">
> 　図：台車型倒立振子（しんし / ふりこ）
>
> $\left\{
\begin{array}{l}
(M+m){x''}
 + (ml\cos\theta){\theta''  }
 -ml\theta'^2\sin\theta = f(t)
 \\
 (ml\cos\theta){x''}
 + (ml^2){\theta''}
 - mlg\sin\theta = 0
\end{array}
\right.$
> * 一次式でない項 $\sin\theta$，$\cos\theta$，$\theta'^2$ があるため，この方程式は数学的に解けない（非線形方程式といいます）
>
> [〔YouTube〕inverted pendulum with mini4WD model car](https://youtu.be/trukC6G44ro)



***
## §２ 微分方程式の数値計算

* 未知数の **微分を含む** 方程式を **微分方程式** といいます．
* 運動方程式は，微分方程式の一種です．
* 運動方程式をコンピュータで解くことを，**（数値）シミュレーション** といいます．

### 《例題１》 $v' = -0.5v$　（水の抵抗を受けるボートの運動方程式）

* 運動方程式より簡単な方程式で，解き方を説明します．
* 未知数が速度 $v(t)$ だけなので，簡単に解けます．

#### 〔運動方程式のプログラミング〕

In [ ]:
def Boat(v, t):
  '''
  《例題１》の運動方程式
  '''
  dvdt = -0.5*v # 運動方程式
  return dvdt

#### 〔時間軸の作成〕

In [ ]:
t0 = 0              # 初期時刻
dt = 0.05           # 時間ステップ
tn = 130            # データ長
t1 = t0 + dt*(tn-1) # 終端時刻

t = np.linspace(t0, t1, tn) # 時間軸を表す等差数列
print(t)

#### 〔初速度の設定〕

In [ ]:
v0 = 1 # 初速度〔m/s〕

#### 〔運動方程式の数値解法〕　速度の時間変化 $v(t)$ を求める

In [ ]:
vt = odeint(Boat, v0, t) # これで解ける

解 $v(t)$ は，時間軸 `t` の各時刻における速度の「数列」として得られる．

| | | | | | |
|:-----|:-----|:-----|:-----|:-----|:-----|
| 時刻の数列 `t` | $0.05$ | $0.1$ | $0.15$ | $0.2$ | $\cdots$ |
| 速度の数列 `v` | $v(0.05)$ | $v(0.1)$ | $v(0.15)$ | $v(0.2)$ | $\cdots$ |


In [ ]:
print(vt)

#### 〔解のグラフ表示〕　（横軸，縦軸）＝（時刻，速度）のグラフ用紙に解を描く

In [ ]:
v = vt[:,0] # 時間軸 t に合せて，解も横ベクトルにしておく

plt.plot(t, v)      # グラフの作成
plt.xlabel('t')     # 横軸のラベル
plt.ylabel('v(t)')  # 縦軸のラベル

#### 〔解であることのチェック〕

$v'=-0.5v$ $\iff$ $v' + 0.5v=0$ が 成立するはず！

In [ ]:
dvdt = np.gradient(v, t, edge_order=2) # v の時間微分

print(dvdt + 0.5*v)

* `e-05` は $\times 10^{-5}$ $(\times 0.00001)$ のコンピュータ表記．
* ゆえに，数値解の誤差は $0.00001$ 程度だったということ．
* 無視できない大きさだが，この実習ではこの程度の誤差でよしとする．

### 《例題１の続き》　位置$x(t)$ も知りたいんですけど！

* 元の運動方程式 $v' = -0.5v$ に，
* 位置と速度の関係式 $x' = v$ を連立します．

 $\left\{
\begin{array}{l}
x'=v
 \\
v' = -0.5v
\end{array}
\right.$

#### 〔運動方程式のプログラミング〕　今度は２連立

In [ ]:
def Boat2(xv, t):
  '''
  《例題１の続き》の運動方程式
  '''
  x, v = xv

  dxdt = v      # 追加した方程式
  dvdt = -0.5*v #《例題１》の運動方程式

  return [dxdt, dvdt] #２式をまとめて返す

#### 〔時間軸の作成〕

In [ ]:
t0 = 0              # 初期時刻
dt = 0.05           # 時間ステップ
tn = 130            # データ長
t1 = t0 + dt*(tn-1) # 終端時刻

t = np.linspace(t0, t1, tn) # 時間軸を表す等差数列
print(t)

#### 〔初期位置と初速度の設定〕

In [ ]:
x0 = 0 #初期位置〔m〕
v0 = 1 #初速度〔m/s〕

#### 〔運動方程式の数値解法〕　位置 $x(t)$ と，速度 $v(t)$ を，同時に求める

In [ ]:
xvt = odeint(Boat2, [x0, v0], t) # これで解ける

解（**数値解** という）は，時間軸 `t` の各時刻における（位置，速度）の「ベクトル列」として得られる．

| | | | | | | |
|:-----|:-----|:-----|:-----|:-----|:-----|:----|
| 時刻の数列 `t` | $0.05$ | $0.1$ | $0.15$ | $0.2$ | $\cdots$ | |
| 解の数列 `xvt` | $x(0.05)$ | $x(0.1)$ | $x(0.15)$ | $x(0.2)$ | $\cdots$ | 位置 |
|  | $v(0.05)$ | $v(0.1)$ | $v(0.15)$ | $v(0.2)$ | $\cdots$ | 速度 |


In [ ]:
print(xvt) # [位置，速度] が時刻毎に縦に並んでいる

#### 〔解のグラフ表示〕

（横軸，縦軸）＝（時刻，位置）

In [ ]:
xt = xvt[:,0] #最初の列（位置）を取り分ける

plt.plot(t, xt) #位置 x(t) グラフのプロット
plt.xlabel('t')  #横軸のラベル
plt.ylabel('x(t)')  #縦軸のラベル

（横軸，縦軸）＝（時刻，速度）　※先ほどと同じグラフ

In [ ]:
vt = xvt[:,1] #次の列（速度）を取り分ける

plt.plot(t, vt) #速度 v(t) グラフのプロット
plt.xlabel('t')  #横軸のラベル
plt.ylabel('v(t)')  #縦軸のラベル

## §３ 数値解のアニメーション

パラパラ漫画方式で，解の動きをアニメーション表示する．

#### 〔アニメーション用のユーザー関数〕

In [ ]:
def animate_Boat(motion):
  '''
  数値解をアニメーション表示する
  '''

  # グラフ用紙の設定
  # fig グラフ用紙
  # ax  座標軸
  fig, ax= plt.subplots(1, 1, figsize=(8,2)) #グラフ用紙(ax)を1行,1列(1枚)用意

  # アニメーション１コマの描画
  def each_frame(i):
    ax.cla() #グラフ用紙を白紙にリセット
    ax.set_xlim(-1,3) #x軸の範囲
    ax.set_ylim(-1,1) #y軸の範囲
    ax.grid()

    # 質点の描画（y方向は0）
    x = motion[i] # x座標
    y = 0         # y座標 存在しないので0
    ax.plot(x, y, 'o')

  # アニメーションデータの作成
  n = len(motion) # コマ数
  anim = FuncAnimation(
      fig,
      each_frame,
      interval=80,
      frames=n
  )

  plt.close()
  return anim

#### 〔アニメーション表示〕

In [ ]:
animate_Boat(xt)

## §４ 大気中の質点の放物運動

 $\left\{
\begin{array}{l}
m x''= - cv|v|
 \\
m y''= - cw|w| - mg
\end{array}
\right.
\quad v = x'
,\quad w = y'
$

####〔空気抵抗のチェック〕


* $F = -cv|v|$ は，速度 $v$ の2乗に比例する空気抵抗
* 抵抗 … 速度と逆向きの力
* 普通の2次関数 $-c v^2$ で与えてしまうと，$v$ の正負によらず，$F$ が同じ向きになり，抵抗にならない．

In [ ]:
def plot_teikou():
  '''
  空気抵抗と2次関数の違い
  '''
  v = np.linspace(-1,1,50) # 速度軸
  c = 0.5 # 仮の抵抗係数

  Q = -c*v*v         # 普通の2次関数
  F = -c*np.abs(v)*v # 空気抵抗

  plt.plot(v, Q, '+', label='$v^2$')
  plt.plot(v, F, label='Air')
  plt.xlabel('v')
  plt.ylabel('F(v)')
  plt.grid()
  plt.legend()

plot_teikou()

* 空気抵抗（実線）は，2次関数のカーブでありつつ，速度と逆向きの特性になっています．

#### 〔運動方程式のプログラミング〕



運動方程式をボートの例と同じ形式に書き直す．2連立 ×（x,y の2方向）＝ 4連立．

 $\left\{
\begin{array}{l}
x' = v
 \\
v'= - \frac{c}{m}v|v|
 \\
y' = w
 \\
w'= - \frac{c}{m}w|w| - g
\end{array}
\quad v = x'
,\quad w = y'
\right.$

In [ ]:
def Shoot(xvyw, t):
  '''
  大気中の放物運動の運動方程式
  '''
  x, v, y, w = xvyw
  m = 1
  c = 0.01 #野球ボールよりだいぶ大きいかも？
  g = 9.8

  # x 方向
  dxdt = v
  dvdt = -(c/m)* v*np.abs(v)

  # y 方向
  dydt = w
  dwdt = -(c/m)* w*np.abs(w) -g

  return [dxdt, dvdt, dydt, dwdt] #４式をまとめて返す

#### 〔運動方程式の数値解法〕

In [ ]:
t = np.linspace(0, 6, 120)

kmph  = 132               # km/h
mps   = kmph*1000 / 60**2 # m/s
angle = np.pi/4           # π/4 = 45度
x0 = 0                    # x方向の初期位置〔m〕
v0 = mps * np.cos(angle)  # x方向の初速度〔m/s〕
y0 = 2                    # y方向の初期位置〔m〕
w0 = mps * np.sin(angle)  # y方向の初速度〔m/s〕

xvywt = odeint(Shoot, [x0, v0, y0, w0], t) # 運動方程式を解く

xt = xvywt[:,0] # x 方向の位置〔m〕
vt = xvywt[:,1] # x 方向の速度〔m/s〕
yt = xvywt[:,2] # y 方向の位置〔m〕
wt = xvywt[:,3] # y 方向の速度〔m/s〕

#### 〔放物運動のアニメーション表示〕

In [ ]:
def animate_Shoot(xt, yt):
  '''
  数値解をアニメーション表示する
  '''
  # グラフ用紙の設定
  # fig グラフ用紙
  # ax  座標軸
  fig, ax= plt.subplots(1, 1, figsize=(5,2)) #グラフ用紙(ax)を1行,1列(1枚)用意

  # アニメーション１コマの描画
  def each_frame(i):
    ax.cla() #グラフ用紙を白紙にリセット
    ax.set_xlim(-1,100) #x軸の範囲
    ax.set_ylim(-1,40) #y軸の範囲
    ax.grid()

    # 運動の描画
    x = xt[i] # x座標
    y = yt[i] # y座標
    ax.plot(x, y, '.r') # 質点
    ax.plot(xt[:i], yt[:i], '-r', lw=0.5) #軌跡

  # アニメーションデータの作成
  n = len(xt) # コマ数
  anim = FuncAnimation(
      fig,
      each_frame,
      interval=80,
      frames=n
  )

  plt.close()
  return anim

In [ ]:
animate_Shoot(xt, yt)

## §５ 運動の設計（≒条件の調整）

* 原因から結果を予測する問題を，**順問題** といいます．
* 逆に，結果を先に決めて，そうなる原因を逆算する問題を，**逆問題** といいます．
* 逆問題を解くことを **設計** といいます．（工学分野の中心的課題）

#### 〔調整作業用のユーザ関数〕

大気中の放物運動を，例に取り上げます．

In [ ]:
def design_Shoot(angle_deg):
  '''
  与えられた角度で，アニメーションまで一括処理する
  '''
  t = np.linspace(0, 7, 140)

  def shoot(deg):

    angle = deg/180 * np.pi # 度→ラジアン

    kmph  = 132               # km/h
    mps   = kmph*1000 / 60**2 # m/s
    x0 = 0                    # x方向の初期位置〔m〕
    v0 = mps * np.cos(angle)  # x方向の初速度〔m/s〕
    y0 = 2                    # y方向の初期位置〔m〕
    w0 = mps * np.sin(angle)  # y方向の初速度〔m/s〕

    xvywt = odeint(Shoot, [x0, v0, y0, w0], t) # 運動方程式を解く

    xt = xvywt[:,0] # x 方向の位置〔m〕
    yt = xvywt[:,2] # y 方向の位置〔m〕

    return [xt, yt]

  xt, yt = shoot( angle_deg )
  xt45, yt45 = shoot( 45 )    # 比較用

  # アニメーション
  fig, ax= plt.subplots(1, 1, figsize=(5,2)) #グラフ用紙(ax)を1行,1列(1枚)用意

  def each_frame(i):
    ax.cla() #グラフ用紙を白紙にリセット
    ax.set_xlim(-1,100) #x軸の範囲
    ax.set_ylim(-1,40) #y軸の範囲

    # 質点の描画
    ax.plot(xt[i], yt[i], '.r', label='new') #質点
    ax.plot(xt[:i], yt[:i], '-r', lw=0.5)    #軌跡

    # 45度の場合
    ax.plot(xt45[i], yt45[i], '.k', label='45') #質点
    ax.plot(xt45[:i], yt45[:i], ':k', lw=0.5)   #軌跡

    ax.grid()
    ax.legend()

  # アニメーションデータの作成
  n = len(xt) # コマ数
  anim = FuncAnimation(
      fig,
      each_frame,
      interval=80,
      frames=n
  )

  plt.close()
  return anim

#### 〔運動の設計〕

**実習**　45度と同等の飛距離で，到達時間の短い射出角度 `angle_deg` を探せ．

In [ ]:
design_Shoot(angle_deg=60) # 45 にすると2つの軌跡が重なります